In [ ]:
import pandas as pd
import numpy as np
import os
import urllib.request
import zipfile
import datetime as dt
from monthdelta import monthdelta
from sqlalchemy import create_engine
from config import username, passphrase


# Define variables
yearMonthList = []
numMonth_lookback = 12
lookback_start_date = dt.date.today()

# Find the past numMonth_lookback YYYYMM
for i in range(0, numMonth_lookback):
    date_ = lookback_start_date - monthdelta(i+1)
    yearMonthList.append(date_.strftime('%Y%m'))
    
# Create a download function
def citibike_downloader(yearMonth):
    
    # Set up url and path
    url = f'https://s3.amazonaws.com/tripdata/{yearMonth}-citibike-tripdata.csv.zip'
    url_old = f'https://s3.amazonaws.com/tripdata/{yearMonth}-citibike-tripdata.zip'
    download_folder = os.getcwd()
    full_path_w_file_name = download_folder + '\\static\\' + f'{yearMonth}-citibike-tripdata.csv.zip'

    # Download CitiBike Zip files
    try:
        urllib.request.urlretrieve(url, full_path_w_file_name)
    except:
        urllib.request.urlretrieve(url_old, full_path_w_file_name)

    # Unzip download files
    zip_ref = zipfile.ZipFile(full_path_w_file_name, 'r')
    zip_ref.extractall(download_folder + '\\static\\')
    zip_ref.close()
    print(f'{yearMonth} data unzipped.')
    
def load_data_to_postgres(yearMonth):
    
    print('Loading ' + yearMonth + '...')

    # Read csv file into pa DataFrame
    data = pd.read_csv(f"static/{yearMonth}-citibike-tripdata.csv")

    # change column names to all lower case
    data.columns = map(str.lower, data.columns)

    # remove all spaces in column names
    data.columns = data.columns.str.replace(' ', '')
    
    # clean up brith year \N
    data.loc[data.birthyear == '\\N', 'birthyear'] = np.nan
    
    # Connect to Postgres
    rds_connection_string = f"{username}:{passphrase}@localhost:5432/citibike"
    engine = create_engine(f'postgresql://{rds_connection_string}')

    # Create table to PG Admin
    data.to_sql(name='tripdata', con=engine, if_exists='append', index=False)
    
    print(yearMonth + ' data loaded to Postgress SQL.')

In [ ]:
# Download all historical data
for yearMonth in yearMonthList:
    citibike_downloader(yearMonth)

In [ ]:
# Load into Postgres SQL
for yearMonth in yearMonthList:
    load_data_to_postgres(yearMonth)